<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Manipulate-Hamiltonian-into-blocks" data-toc-modified-id="Manipulate-Hamiltonian-into-blocks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Manipulate Hamiltonian into blocks</a></span></li><li><span><a href="#Feature-computation" data-toc-modified-id="Feature-computation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature computation</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Restructuring-blocks-of-the-Hamiltonian" data-toc-modified-id="Restructuring-blocks-of-the-Hamiltonian-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Restructuring blocks of the Hamiltonian</a></span></li><li><span><a href="#OLD-AND-EXISTING" data-toc-modified-id="OLD-AND-EXISTING-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>OLD AND EXISTING</a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import json
from ase.io import read
from itertools import product
import matplotlib.pyplot as plt
# from rascal.representations import SphericalExpansion
import copy
from ase.units import Hartree

from torch_hamiltonian_utils.torch_cg import ClebschGordanReal
from torch_hamiltonian_utils.torch_hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks, hamiltonian_features
from torch_hamiltonian_utils.torch_builder import TensorBuilder

from equistore import Labels, TensorBlock, TensorMap
from equistore_utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from equistore_utils.acdc_mini import acdc_standardize_keys, cg_increment, cg_combine
from equistore_utils.model_hamiltonian import get_feat_keys 

import importlib
torch.set_default_dtype(torch.float64)

In [3]:
from equistore_utils.model_hamiltonian import get_feat_keys_from_uncoupled


In [4]:
frames1 = read("data/water-hamiltonian/water_coords_1000.xyz",":2")
frames2 =  read("data/ethanol-hamiltonian/ethanol_4500.xyz", ":2")
frames = frames1 #+ frames2
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50
    
jorbs = json.loads(json.load(open('data/water-hamiltonian/water_orbs.json', "r")))
# jorbs = json.load(open('data/ethanol-hamiltonian/orbs_saph_ethanol.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1, "C":6}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]
focks1 = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)[:len(frames1)]
focks2 = np.load("data/ethanol-hamiltonian/ethanol_saph_orthogonal.npy", allow_pickle = True)[:len(frames2)]
focks = focks1

In [5]:
##generating jagged matrix

# f=[]
# for x in focks1:
#     f.append(x)
# for x in focks2:
#     f.append(x) 
# focks = np.asanyarray(f, dtype=object)
# for f in focks: 
#     print(f.shape)
# dense_to_blocks(focks, frames, orbs)

In [6]:
cg = ClebschGordanReal(4)

## Manipulate Hamiltonian into blocks

In [467]:
blocks = dense_to_blocks(focks, frames, orbs)
fock_bc = couple_blocks(blocks, cg)

In [468]:
frames

[Atoms(symbols='C2OH6', pbc=False, cell=[100.0, 100.0, 100.0]),
 Atoms(symbols='C2OH6', pbc=False, cell=[100.0, 100.0, 100.0])]

In [469]:
print("blockidx", blocks.blocks_matching(block_type=1))
print("keys", blocks.keys[blocks.blocks_matching(block_type=1)])

blockidx [3, 4, 5, 18]
keys [(1, 6, 2, 0, 6, 2, 0) (1, 6, 2, 0, 6, 2, 1) (1, 6, 2, 1, 6, 2, 1)
 (1, 1, 1, 0, 1, 1, 0)]


In [431]:
get_feat_keys_from_uncoupled(blocks.keys[2],order_nu=2)

Labels([(2,  1, 0, 8, 8, 0), (2, -1, 1, 8, 8, 0), (2,  1, 2, 8, 8, 0)],
       dtype=[('order_nu', '<i4'), ('inversion_sigma', '<i4'), ('spherical_harmonics_l', '<i4'), ('species_center', '<i4'), ('species_neighbor', '<i4'), ('block_type', '<i4')])

In [432]:
fock_bc.block(0).values

tensor([[[-0.8332]],

        [[-0.7726]]])

## Feature computation

In [433]:
rascal_hypers = {
    "interaction_cutoff": 4.0,
    "cutoff_smooth_width": 0.5,
    "max_radial": 4,
    "max_angular": 2,
    "gaussian_sigma_constant" : 0.2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

In [434]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [435]:
rho1i = acdc_standardize_keys(rhoi)
rho1i.keys_to_properties(['species_neighbor'])
gij =  acdc_standardize_keys(gij)

In [436]:
rho2i = cg_increment(rho1i, rho1i, lcut=3, other_keys_match=["species_center"], clebsch_gordan=cg)

In [437]:
#rho3i = cg_increment(rho2i, rho1i, lcut=2, other_keys_match=["species_center"], clebsch_gordan=cg)

In [438]:
rho1ij = cg_increment(rho1i, gij, lcut=3, other_keys_match=["species_center"], clebsch_gordan=cg)

In [439]:
#rho2ij = cg_increment(rho2i, gij, lcut=2, other_keys_match=["species_center"], clebsch_gordan=cg)

In [440]:
features = hamiltonian_features(rho2i, rho1ij)

## Model 

In [441]:
idx_used = [0,3,4,5,9,14,15,16,19]
for i in range(len(features.keys)):
    if i in idx_used:
        continue
    print(i,features.keys[i])
#checked keys of features not used - makes sense - either they are too high L or use inv_sigma thats not right 


1 (2, 1, 1, 1, 1, 0)
2 (2, 1, 2, 1, 1, 0)
6 (2, -1, 1, 1, 1, 0)
7 (2, -1, 2, 1, 1, 0)
8 (2, 1, 3, 1, 1, 0)
10 (2, -1, 2, 8, 8, 0)
11 (2, 1, 3, 8, 8, 0)
12 (2, -1, 3, 1, 1, 0)
13 (2, -1, 3, 8, 8, 0)
17 (2, 1, 1, 1, 1, 1)
18 (2, 1, 1, 1, 1, -1)
20 (2, 1, 2, 1, 1, 1)
21 (2, 1, 2, 1, 1, -1)
22 (2, 1, 2, 1, 8, 2)
23 (2, -1, 1, 1, 1, 1)
24 (2, -1, 1, 1, 1, -1)
25 (2, -1, 1, 1, 8, 2)
26 (2, -1, 2, 1, 1, 1)
27 (2, -1, 2, 1, 1, -1)
28 (2, 1, 3, 1, 1, 1)
29 (2, 1, 3, 1, 1, -1)
30 (2, -1, 2, 1, 8, 2)
31 (2, 1, 3, 1, 8, 2)
32 (2, -1, 3, 1, 1, 1)
33 (2, -1, 3, 1, 1, -1)
34 (2, -1, 3, 1, 8, 2)


In [442]:
class HamModel(torch.nn.ModuleDict):
    def __init__(self, frames, feature, target, regularization=None, seed=None, layer_size=None):
        super().__init__()
        self.frames= frames
        self.features = features 
        self.target = target
        self.models={}
        for key, block in target:        
            blocktype, L, nu,sigma,species1, species2 = get_feat_key(key)
            block_feat = self.features.block(block_type=blocktype, spherical_harmonics_l=L, inversion_sigma=sigma, 
                                       species_center=species1, species_neighbor=species2)
            
            self.models[(key)] = BlockModel(block, block_feat, layer_size) 
            self.models[(key)].initialize_model(seed)
            
            
    def forward(self, target):
        pred_blocks =[]
        for key, block in target:
            pred = self.models[str(key)].forward(block)
            newblock = TensorBlock(
                        values=pred.reshape((-1, 1, 1)),
                        samples=block.samples,
                        components=block.components,
                        properties= Labels(["dummy"], np.asarray([[0]], dtype=np.int32))
                    )
            pred_blocks.append(newblock) 
        
        keys = target.keys
        pred_target = TensorMap(keys, pred_blocks)
        return(pred_target)
    
    def train_individual(self, train_dataloader_dict, optimizer_type, n_epochs, loss, lr):
        #Iterates through the keys of self.model, then for each key we will fit self.model[key] with data[key]
        for key in self.models:
            print ("Now training model {} of {}".format(current, total))
            #model[key].fit()
            pass
    def train_collective(optimizer_type):
        data
        opt = LBFGS
        for all keys:
            for data[key]:
                pred_block = model[key](data[key])
                
        total block = 
        loss()
        reg_loss
        backward

SyntaxError: invalid syntax (2839764941.py, line 42)

In [71]:
testmodel = BlockModel([10,20,30], [5,3,1], "cpu", keys = ['a', 'b', 'c'])
weight = list(testmodel.parameters())

In [82]:
a = {}
a[(1,0,1,1)] = 5

In [52]:
class BlockModel(torch.nn.Module): #Currently only 1 model per block
    def __init__(self, inputSize, outputSize, device, keys, seed = None, hidden_layers = None):
        super().__init__()
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.device = device
        self.keys = keys
        self.hidden_layers = hidden_layers
        self.initialize_model(seed)
        
        self.to(device)
    
    def initialize_model(self, seed):
        
        if seed is not None:
            torch.manual_seed(seed)
        
        self.model = torch.nn.ModuleDict()
        for index, key in enumerate(self.keys):
            self.model[key] = torch.nn.Linear(self.inputSize[index], self.outputSize[index], bias = False)
        else:
            if self.hidden_layers is None: 
                self.nn = torch.nn.Linear(self.inputSize, self.outputSize, bias = False)
            
            else:
                print ("DNN not supported yet")            
#                 self.layer1 = nn.Linear(10,10)
#                 self.act1 = nn.TanH()
#                 self.layer2 = nn.Linear(10,100)
#                 self.act2 = nn.Sigmoid()
#                 self.nn = torch.nn.Sequential(
#                     linear_init_weights(inputSize, self.layer_size),
#                     torch.nn.Tanh(),
#                     linear_init_weights(self.layer_size, self.layer_size),
#                     torch.nn.Tanh(),
#                     linear_init_weights(self.layer_size, OutputSize),
#                 )
           
        
    def forward(self, x):
        pred_values = []
        for i, key in enumerate(x.key):
            pred_values.append(self.model[key](x[key]))
        pred_values.hstack???
        pred = self.nn(x)
#         pred = self.layer1(pred)
#         pred = self.act1(pred)
#         pred = self.layer2(pred)
#         pred = self.act2(pred)
#         pred = self.layer1(pred)
        return pred 

    def fit(traindata_loader, loss_function, optimizer_type, lr, reg, nepochs = 5000):
        if optimizer_type == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
            threshold = 200
            scheduler_threshold = 200
            tol = 1e-2
            history_step = 1000
        
        elif optimizer_type == "LBFGS":
            optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)
            threshold = 30
            scheduler_threshold = 30
            tol = 1e-2
            history_step = 10
            
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, scheduler_threshold, gamma = 0.5)
        best_state = copy.deepcopy(self.state_dict())
        lowest_loss = torch.tensor(9999)
        pred_loss = torch.tensor(0)
        trigger = 0
        loss_history = []
        pbar = tqdm(range(npochs))
        
        for epoch in pbar:
            pbar.set_description(f"Epoch: {epoch}")
            pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)
            
            for x_data, y_data in traindata_loader: 
                optimizer.zero_grad()
                x_data, y_data = x_data.to(self.device), y_data.to(self.device)
                if optimizer_type == "LBFGS":
                    def closure():
                        optimizer.zero_grad()
                        _pred = self.forward(x_data)
                        _pred_loss = loss_function(_pred, y_data)       
                        _pred_loss = torch.nan_to_num(_pred_loss, nan=lowest_loss.item(), posinf = lowest_loss.item(), neginf = lowest_loss.item())                 
                        _reg_loss = torch.sum(torch.pow(self.nn.weight,2)) #Only works for 1 layer
                        _reg_loss *= reg.item() #Might only work for 1 layer
                        _new_loss = _pred_loss + _reg_loss
                        _new_loss.backward()
                        return _new_loss
                    opt.step(closure)
                elif optimizer_type == "Adam":
                    pred = self.forward(x_data)
                    pred_loss = loss_function(pred, y_data)
                    reg_loss = torch.sum(torch.pow(self.nn.weight,2))#Only works for 1 layer
                    new_loss = pred_loss + reg_loss
                    new_loss.backward()
                    optimizer.step()
                
            with torch.no_grad():
                current_loss = 0 
                for x_data, y_data in traindata_loader:
                    pred = self.forward(self.feature.values)
                    current_loss  += loss_function(pred, y_data) #Loss should be normalized already
                pred_loss = current_loss
                reg_loss = torch.sum(torch.pow(self.nn.weight,2))#Only works for 1 layer
                new_loss = pred_loss + reg_loss
                
                if pred_loss >100000 or (pred_loss.isnan().any()) :
                    print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
                    self.load_state_dict(best_state)
                    if optimizer_type == "Adam":
                        optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
                    elif optimizer_type == "LBFGS":
                        optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)

                if epoch % history_step == 1:
                    loss_history.append(lowest_loss.item())
                
                if lowest_loss - new_loss > tol: #threshold to stop training
                    best_state = copy.deepcopy(self.state_dict())
                    lowest_loss = new_loss 
                    trigger = 0 
                    
                    
                else:
                    trigger += 1
                    scheduler.step()
                    if trigger > threshold:
                        self.load_state_dict(best_state)
                        print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
                        return loss_history
        return loss_history

## Restructuring blocks of the Hamiltonian 

We need to merge blocktype +1 and -1 together as components 

In [7]:
frames1 = read("data/water-hamiltonian/water_coords_1000.xyz",":2")
frames2 =  read("data/ethanol-hamiltonian/ethanol_4500.xyz", ":2")
frames = frames2 #+ frames2
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50
    
# jorbs = json.loads(json.load(open('data/water-hamiltonian/water_orbs.json', "r")))
jorbs = json.load(open('data/ethanol-hamiltonian/orbs_saph_ethanol.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1, "C":6}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]
focks1 = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)[:len(frames1)]
focks2 = np.load("data/ethanol-hamiltonian/ethanol_saph_orthogonal.npy", allow_pickle = True)[:len(frames2)]
focks = focks2

In [112]:
from torch_hamiltonian_utils.torch_hamiltonians import _orbs_offsets, _components_idx, _components_idx_2d
def couple_blocks(blocks, cg=None):
    if cg is None:
        lmax = max(blocks.keys["li"]+blocks.keys["lj"])
        cg = ClebschGordanReal(lmax)

    block_builder = TensorBuilder(["block_type", "a_i", "n_i", "l_i", "a_j", "n_j", "l_j", "L"], ["structure", "center", "neighbor"], [["M"]], ["value"])
    for idx, block in blocks:
        block_type, ai, ni, li, aj, nj, lj = tuple(idx)
        decoupled = torch.moveaxis(block.values,-1,-2).reshape((len(block.samples), len(block.properties), 2*li+1, 2*lj+1))
        coupled = cg.couple(decoupled)[(li,lj)]
        for L in coupled:
            bsamples = list(np.array(block.samples.asarray(), dtype=int))
            samples_keep = list(range(len(bsamples)))
            block_idx = tuple(idx) + (L,)
            # skip blocks that are zero because of symmetry
            if ai==aj and ni==nj and li==lj:
                parity = (-1)**(li+lj+L)
                if (parity == -1 and block_type ==0):
                    continue
                elif block_type == 1:
                    if parity==-1:
                        #remove samples with A>0 
                        sample_idx=np.where(block.samples["structure"]>0)[0]
                        coupled[L][sample_idx] = 0

                    else:
                        #remove samples with A<0
                        sample_idx=np.where(block.samples["structure"]<0)[0]
                        coupled[L][sample_idx] = 0
#                         for i in sorted(sample_idx, reverse=True):
#                             bsamples.pop(i)
#                             samples_keep.pop(i)
                        
            new_block = block_builder.add_block(keys=block_idx, properties=np.asarray([[0]], dtype=np.int32), 
                            components=[_components_idx(L).reshape(-1,1)] )
                
            new_block.add_samples(labels=np.asarray(bsamples,dtype=np.int32).reshape(len(bsamples),-1), 
                            data=torch.moveaxis(coupled[L][samples_keep], -1, -2 ) )

    return block_builder.build()

def decouple_blocks(blocks, cg=None):
    if cg is None:
        lmax = max(blocks.keys["L"])
        cg = ClebschGordanReal(lmax)

    block_builder = TensorBuilder(["block_type", "a_i", "n_i", "l_i", "a_j", "n_j", "l_j"], ["structure", "center", "neighbor"], [["m1"], ["m2"]], ["value"])
    for idx, block in blocks:
        block_type, ai, ni, li, aj, nj, lj, L = tuple(idx)
        block_idx = (block_type, ai, ni, li, aj, nj, lj)
        bsamples=[]
        if block_idx in block_builder.blocks:
            continue        
        coupled = {}
        for L in range(np.abs(li-lj), li+lj+1):
            bidx = blocks.keys.position(block_idx+(L,))
            if bidx is not None:
#                 print(idx, blocks.block(bidx).values.shape, torch.moveaxis(blocks.block(bidx).values, -1, -2) .shape)
                coupled[L] = torch.moveaxis(blocks.block(bidx).values, -1, -2) 
                bsamples.append(blocks.block(bidx).samples)
        decoupled = cg.decouple( {(li,lj):coupled})
        
        new_block = block_builder.add_block(keys=block_idx, properties=np.asarray([[0]], dtype=np.int32), 
                            components=[_components_idx(li), _components_idx(lj)] )
        new_block.add_samples(labels=block.samples.view(dtype=np.int32).reshape(block.samples.shape[0],-1),
                            data=torch.moveaxis(decoupled, 1, -1))
    return block_builder.build()    


In [149]:
fock_bc.blocks_matching(block_type=1)

[4, 5, 6, 7, 8, 24]

In [161]:
fock_bc.block(24).samples#values

Labels([(1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 3, 7), (1, 3, 8), (1, 4, 5),
        (1, 4, 6), (1, 4, 7), (1, 4, 8), (1, 5, 6), (1, 5, 7), (1, 5, 8),
        (1, 6, 7), (1, 6, 8), (1, 7, 8), (2, 3, 4), (2, 3, 5), (2, 3, 6),
        (2, 3, 7), (2, 3, 8), (2, 4, 5), (2, 4, 6), (2, 4, 7), (2, 4, 8),
        (2, 5, 6), (2, 5, 7), (2, 5, 8), (2, 6, 7), (2, 6, 8), (2, 7, 8)],
       dtype=[('structure', '<i4'), ('center', '<i4'), ('neighbor', '<i4')])

In [163]:
len(range(np.abs(1), 3))

2

In [250]:
from torch_hamiltonian_utils.torch_hamiltonians import _orbs_offsets, _components_idx, _components_idx_2d
def couple_blocks(blocks, cg=None):
    if cg is None:
        lmax = max(blocks.keys["li"]+blocks.keys["lj"])
        cg = ClebschGordanReal(lmax)

    block_builder = TensorBuilder(["block_type", "a_i", "n_i", "l_i", "a_j", "n_j", "l_j", "L"], ["structure", "center", "neighbor"], [["M"]], ["value"])
    for idx, block in blocks:
        block_type, ai, ni, li, aj, nj, lj = tuple(idx)
        decoupled = torch.moveaxis(block.values,-1,-2).reshape((len(block.samples), len(block.properties), 2*li+1, 2*lj+1))
        coupled = cg.couple(decoupled)[(li,lj)]
        for L in coupled:
            bsamples = list(np.array(block.samples.asarray(), dtype=int))
            samples_keep = list(range(len(bsamples)))
            block_idx = tuple(idx) + (L,)
            # skip blocks that are zero because of symmetry
            if ai==aj and ni==nj and li==lj:
                parity = (-1)**(li+lj+L)
                if (parity == -1 and block_type ==0):
                    continue
                elif block_type == 1:
                    if parity==-1:
                        #remove samples with A>0 
                        sample_idx=np.where(block.samples["structure"]>0)[0]
                        for i in sorted(sample_idx, reverse=True):
                            bsamples.pop(i)
                            samples_keep.pop(i)

                    else:
                        #parity =1 and remove samples with A<0
                        sample_idx=np.where(block.samples["structure"]<0)[0]
                        for i in sorted(sample_idx, reverse=True):
                            bsamples.pop(i)
                            samples_keep.pop(i)
                        
            new_block = block_builder.add_block(keys=block_idx, properties=np.asarray([[0]], dtype=np.int32), 
                            components=[_components_idx(L).reshape(-1,1)] )
                
            new_block.add_samples(labels=np.asarray(bsamples,dtype=np.int32).reshape(len(bsamples),-1), 
                            data=torch.moveaxis(coupled[L][samples_keep], -1, -2 ) )

    return block_builder.build()

def decouple_blocks(blocks, cg=None):
    #append and add to samples carefully  in coupled[L] - otherwise decoupled will screw up 
    if cg is None:
        lmax = max(blocks.keys["L"])
        cg = ClebschGordanReal(lmax)
    block_builder = TensorBuilder(["block_type", "a_i", "n_i", "l_i", "a_j", "n_j", "l_j"], ["structure", "center", "neighbor"], [["m1"], ["m2"]], ["value"])
   
    for idx, block in blocks:
        block_type, ai, ni, li, aj, nj, lj, L = tuple(idx)
        block_idx = (block_type, ai, ni, li, aj, nj, lj)
        bsamples = block.samples
        
        if block_type==1 and ni==nj and li==lj and len(range(np.abs(li-lj), li+lj+1))>1: 
            s1 = blocks.block(blocks.keys.position(block_idx+(np.abs(li-lj),))).samples 
            s2 = s1.copy()
            s2["structure"]=s2["structure"]*(-1)
            if s1["structure"][0]>0 :
                even_sample = s1
                odd_sample = s2
            else: 
                even_sample = s2
                odd_sample = s1
            bsamples = np.zeros((2*len(s1),), dtype = s1.dtype)
            bsamples[0::2] = even_sample
            bsamples[1::2] = odd_sample
            
        if block_idx in block_builder.blocks:
            continue        
        coupled = {}
        for L in range(np.abs(li-lj), li+lj+1):
            bidx = blocks.keys.position(block_idx+(L,))
            if bidx is not None:
                coupled[L] = torch.moveaxis(blocks.block(bidx).values, -1, -2)
                if block_type==1 and ni==nj and li==lj and len(range(np.abs(li-lj), li+lj+1))>1: 
                    coupled[L] = torch.zeros(len(bsamples), 1, 2*L+1)
                    if blocks.block(bidx).samples["structure"][0]>0:
                        print(idx, coupled[L].shape, torch.moveaxis(blocks.block(bidx).values, -1, -2).shape, bsamples.shape)
                        coupled[L][0::2] =  torch.moveaxis(blocks.block(bidx).values, -1, -2)
                    else: 
                        coupled[L][1::2] =  torch.moveaxis(blocks.block(bidx).values, -1, -2)


        decoupled = cg.decouple({(li,lj):coupled})
        
        new_block = block_builder.add_block(keys=block_idx, properties=np.asarray([[0]], dtype=np.int32), 
                            components=[_components_idx(li), _components_idx(lj)] )
        new_block.add_samples(labels=bsamples.view(dtype=np.int32).reshape(bsamples.shape[0],-1),
                            data=torch.moveaxis(decoupled, 1, -1))
    return block_builder.build()    


In [268]:
from torch_hamiltonian_utils.torch_hamiltonians import _orbs_offsets, _components_idx, _components_idx_2d, _atom_blocks_idx
def blocks_to_dense(blocks, frames, orbs):
    """
    Converts a TensorMap containing matrix blocks in the uncoupled basis, `blocks` into dense matrices.
    Needs `frames` and `orbs` to reconstruct matrices in the correct order. See `dense_to_blocks` to understant
    the different types of blocks.
    """

    orbs_tot, orbs_offset =  _orbs_offsets(orbs)
    
    atom_blocks_idx = _atom_blocks_idx(frames, orbs_tot)
    
    # init storage for the dense hamiltonians
    dense = []        
    for f in frames:
        norbs = 0
        for ai in f.numbers:
            norbs += orbs_tot[ai]
        ham = torch.zeros(norbs, norbs)#, dtype=np.float64)
        dense.append(ham)

    # loops over block types
    for idx, block in blocks:
        cur_struct = 9999
        block_type, ai, ni, li, aj, nj, lj = tuple(idx)

        # offset of the orbital block within the pair block in the matrix
        ki_offset = orbs_offset[(ai,ni,li)]
        kj_offset = orbs_offset[(aj,nj,lj)]
        
        # loops over samples (structure, i, j)
        
        for (A,i,j), block_data in zip(block.samples, block.values): 
            if A<0:
                    continue 
            if A != cur_struct:
                cur_struct = A - 1
                ham = dense[cur_struct]
            # coordinates of the atom block in the matrix
            ki_base, kj_base = atom_blocks_idx[(cur_struct ,i,j)]
            islice = slice(ki_base+ki_offset, ki_base+ki_offset+2*li+1)
            jslice = slice(kj_base+kj_offset, kj_base+kj_offset+2*lj+1)

            # print(i, ni, li, ki_base, ki_offset)
            if block_type == 0:
                ham[islice, jslice] = block_data[:,:,0].reshape(2*li+1,2*lj+1)
                if ki_offset != kj_offset:
                    ham[jslice, islice] = block_data[:,:,0].reshape(2*li+1,2*lj+1).T
            elif block_type == 2:
                ham[islice, jslice] = block_data[:,:,0].reshape(2*li+1,2*lj+1)
                ham[jslice, islice] = block_data[:,:,0].reshape(2*li+1,2*lj+1).T   
                
            elif block_type == 1:
                block_idx_minus = np.where(np.logical_and(np.logical_and(block.samples["structure"]==-A,  block.samples["center"]==i), block.samples["neighbor"]==j))
#                 print(idx, block_idx_minus, block.values[block_idx_minus].shape, li, lj)
#                 print(block.samples[block_idx_minus], A, i, j)
                if block_idx_minus[0]:
                    print(block_idx_minus)
                    assert block.values[block_idx_minus][0,:,:,0].shape == block_data[:,:,0].shape
                    block_data_minus = block.values[block_idx_minus][0,:,:,0].reshape(2*li+1,2*lj+1)  / np.sqrt(2)
                else: 
                    block_data_minus = np.zeros_like(block_data)
                block_data_plus = block_data[:,:,0].reshape(2*li+1,2*lj+1)  / np.sqrt(2)
#                 print(cur_struct , block_data.shape)
                ham[islice, jslice] +=  block_data_plus
                ham[jslice, islice] +=  block_data_plus.T
                ham[islice, jslice] += block_data_minus
                ham[jslice, islice] += block_data_minus.T
                if ki_offset != kj_offset:
                    islice = slice(ki_base+kj_offset, ki_base+kj_offset+2*lj+1)
                    jslice = slice(kj_base+ki_offset, kj_base+ki_offset+2*li+1)
                    
                    ham[islice, jslice] += block_data_plus.T
                    ham[jslice, islice] += block_data_plus
                    
                    ham[islice, jslice] -= block_data_minus .T
                    ham[jslice, islice] -= block_data_minus
                
                    
    return dense

In [271]:
blocks = dense_to_blocks(focks, frames, orbs)
fock_bc = couple_blocks(blocks, cg)
fock_dc = decouple_blocks(fock_bc, cg)
r = blocks_to_dense(fock_dc, frames, orbs)
rp = blocks_to_dense(blocks, frames, orbs)

(1, 6, 2, 1, 6, 2, 1, 0) torch.Size([4, 1, 1]) torch.Size([2, 1, 1]) (4,)
(1, 6, 2, 1, 6, 2, 1, 0) torch.Size([4, 1, 5]) torch.Size([2, 1, 5]) (4,)
(array([1]),)
(array([3]),)
(array([1]),)
(array([3]),)
(array([1]),)
(array([3]),)
(array([1]),)
(array([3]),)
(array([1]),)
(array([3]),)
(array([1]),)
(array([3]),)
(array([5]),)
(array([7]),)
(array([9]),)
(array([11]),)
(array([13]),)
(array([15]),)
(array([17]),)
(array([19]),)
(array([21]),)
(array([23]),)
(array([25]),)
(array([27]),)
(array([29]),)
(array([31]),)
(array([33]),)
(array([35]),)
(array([37]),)
(array([39]),)
(array([41]),)
(array([43]),)
(array([45]),)
(array([47]),)
(array([49]),)
(array([51]),)
(array([53]),)
(array([55]),)
(array([57]),)
(array([59]),)


/var/folders/81/mvs1jcdn4871nrddg_30v9b40000gn/T/ipykernel_92048/998866105.py:57: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if block_idx_minus[0]:


In [272]:
cc= couple_blocks(fock_dc, cg)

In [274]:
for i, f in enumerate(focks):
    print(i, np.linalg.norm(r[i] - focks[i]))
    print(i, np.linalg.norm(rp[i] - focks[i]))

0 1.3716912657634958e-07
0 1.3384690121057745e-07
1 1.4969581206791603e-07
1 1.4666266862600315e-07


In [110]:
fock_bc2.block(block_type=-1, L=1).samples#values

NameError: name 'fock_bc2' is not defined

In [38]:
from torch_hamiltonian_utils.torch_hamiltonians import old_dense_to_blocks, old_blocks_to_dense
blocks2 = old_dense_to_blocks(focks, frames, orbs)
fock_bc2 = couple_blocks(blocks2, cg)
fock_dc2 = decouple_blocks(fock_bc2, cg)
r2= old_blocks_to_dense(fock_dc2, frames, orbs)
rp2 = old_blocks_to_dense(blocks2, frames, orbs)

(0, 6, 2, 0, 6, 2, 0, 0) torch.Size([4, 1, 1]) torch.Size([4, 1, 1])
(0, 6, 2, 0, 6, 2, 1, 1) torch.Size([4, 3, 1]) torch.Size([4, 1, 3])
(0, 6, 2, 1, 6, 2, 1, 0) torch.Size([4, 1, 1]) torch.Size([4, 1, 1])
(0, 6, 2, 1, 6, 2, 1, 0) torch.Size([4, 5, 1]) torch.Size([4, 1, 5])
(1, 6, 2, 0, 6, 2, 0, 0) torch.Size([2, 1, 1]) torch.Size([2, 1, 1])
(1, 6, 2, 0, 6, 2, 1, 1) torch.Size([2, 3, 1]) torch.Size([2, 1, 3])
(-1, 6, 2, 0, 6, 2, 1, 1) torch.Size([2, 3, 1]) torch.Size([2, 1, 3])
(1, 6, 2, 1, 6, 2, 1, 0) torch.Size([2, 1, 1]) torch.Size([2, 1, 1])
(1, 6, 2, 1, 6, 2, 1, 0) torch.Size([2, 5, 1]) torch.Size([2, 1, 5])
(-1, 6, 2, 1, 6, 2, 1, 1) torch.Size([2, 3, 1]) torch.Size([2, 1, 3])
(2, 6, 2, 0, 8, 2, 0, 0) torch.Size([4, 1, 1]) torch.Size([4, 1, 1])
(2, 6, 2, 0, 8, 2, 1, 1) torch.Size([4, 3, 1]) torch.Size([4, 1, 3])
(2, 6, 2, 1, 8, 2, 0, 1) torch.Size([4, 3, 1]) torch.Size([4, 1, 3])
(2, 6, 2, 1, 8, 2, 1, 0) torch.Size([4, 1, 1]) torch.Size([4, 1, 1])
(2, 6, 2, 1, 8, 2, 1, 0) torch.S

In [39]:
for k, b in fock_dc2:
    if np.linalg.norm(blocks2.block(k).values - b.values)>1e-7:
        print(k, np.linalg.norm(blocks2.block(k).values - b.values))

In [41]:
for i, f in enumerate(focks):
    print(i, np.linalg.norm(r2[i] - focks[i]))

0 1.3716912657634958e-07
1 1.4969581206791603e-07


In [79]:
for k, b in fock_bc:
    n = len(fock_bc2.block(k).values)
    if k["block_type"]==1:
        print(k,np.linalg.norm(fock_bc2.block(k).values - b.values[::2]))
        mkey = Labels(k.dtype.names, np.asarray([(-1, k[1], k[2], k[3], k[4], k[5], k[6], k[7])], np.int32))
        if fock_bc2.keys.position(mkey[0]):
            print(mkey,np.linalg.norm(fock_bc2.block(mkey).values - b.values[1::2]) )

for k, b in fock_dc:
    n = len(fock_dc2.block(k).values)
    if k["block_type"]==1:
        print(k,np.linalg.norm(fock_dc2.block(k).values - b.values[::2]))
        mkey = Labels(k.dtype.names, np.asarray([(-1, k[1], k[2], k[3], k[4], k[5], k[6])], np.int32))
        if fock_dc2.keys.position(mkey[0]):
            print(np.linalg.norm(fock_dc2.block(mkey).values), np.linalg.norm(b.values[1::2]) )
            print(mkey,np.linalg.norm(fock_dc2.block(mkey).values - b.values[1::2]) )

    

(1, 6, 2, 0, 6, 2, 0, 0) 0.0
(1, 6, 2, 0, 6, 2, 1, 1) 0.0
[(-1, 6, 2, 0, 6, 2, 1, 1)] 0.0
(1, 6, 2, 1, 6, 2, 1, 0) 0.0
(1, 6, 2, 1, 6, 2, 1, 2) 0.0
(1, 1, 1, 0, 1, 1, 0, 0) 0.0
(1, 6, 2, 0, 6, 2, 0) 0.0
(1, 6, 2, 0, 6, 2, 1) 0.0
0.5378022878260719 0.5378022878260719
[(-1, 6, 2, 0, 6, 2, 1)] 0.0
(1, 6, 2, 1, 6, 2, 1) 0.0
0.02060351348093143 0.0
[(-1, 6, 2, 1, 6, 2, 1)] 0.02060351348093143
(1, 1, 1, 0, 1, 1, 0) 0.0


In [455]:
?decouple_blocks

In [32]:
blocks.keys.dtype.names

('block_type', 'a_i', 'n_i', 'l_i', 'a_j', 'n_j', 'l_j')

## OLD AND EXISTING

In [181]:

#------------------------------------------------
#Soln 1 - have the same number of components - so add a redundant dimension of perm_sym across ALL blocks -this code
#--------------------------------------------------
from torch_hamiltonian_utils.torch_hamiltonians import _orbs_offsets, _components_idx, _components_idx_2d

def dense_to_blocks(dense, frames, orbs):
    """
    Converts a list of dense matrices `dense` corresponding to the single-particle Hamiltonians for the structures
    described by `frames`, and using the orbitals described in the dictionary `orbs` into a TensorMap storage format.

    The label convention is as follows: 

    The keys that label the blocks are ["block_type", "a_i", "n_i", "l_i", "a_j", "n_j", "l_j"].
    block_type: 0 -> diagonal blocks, atom i=j
                2 -> different species block, stores only when n_i,l_i and n_j,l_j are lexicographically sorted
                1,-1 -> same specie, off-diagonal. store separately symmetric (1) and anti-symmetric (-1) term
    a_{i,j}: chemical species (atomic number) of the two atoms
    n_{i,j}: radial channel
    l_{i,j}: angular momentum
    """

    block_builder = TensorBuilder(["block_type", "a_i", "n_i", "l_i", "a_j", "n_j", "l_j"], ["structure", "center", "neighbor"], [["m1"], ["m2"], ["perm_sym"]], ["value"])
    orbs_tot, _ = _orbs_offsets(orbs)
    for A in range(len(frames)):
        frame = frames[A]
        ham = dense[A]
        ki_base = 0
        for i, ai in enumerate(frame.numbers):
            kj_base = 0
            for j, aj in enumerate(frame.numbers):
                if i==j:
                    block_type = 0  # diagonal
                elif ai==aj:
                    block_type = 1  # same-species
                    if i>j:
                        kj_base+=orbs_tot[aj]
                        continue
                else:
                    block_type = 2  # different species
                    if ai>aj: # only sorted element types
                        kj_base += orbs_tot[aj]
                        continue
                block_data = torch.from_numpy(ham[ki_base:ki_base+orbs_tot[ai], kj_base:kj_base+orbs_tot[aj]])
                
                if block_type == 1:
                    block_data_plus = (block_data + block_data.T) *1/np.sqrt(2)
                    block_data_minus = (block_data - block_data.T) *1/np.sqrt(2)
                ki_offset = 0
                for ni, li, mi in orbs[ai]:
                    if mi != -li: # picks the beginning of each (n,l) block and skips the other orbitals
                        continue
                    kj_offset = 0
                    for nj, lj, mj in orbs[aj]:
                        if mj != -lj: # picks the beginning of each (n,l) block and skips the other orbitals
                            continue                    
                        if (ai==aj and (ni>nj or (ni==nj and li>lj))): 
                            kj_offset += 2*lj+1
                            continue
                        block_idx = (block_type, ai, ni, li, aj, nj, lj)
                        if block_idx not in block_builder.blocks:
                            block = block_builder.add_block(keys=block_idx, properties=np.asarray([[0]], dtype=np.int32),
                                            components=[_components_idx(li), _components_idx(lj), np.array([+1,-1], np.int32).reshape(-1,1)] ) 
#                             if block_type %2 ==0:
#                                 block = block_builder.add_block(keys=block_idx, properties=np.asarray([[0]], dtype=np.int32),
#                                             components=[_components_idx(li), _components_idx(lj)] )
                           
                            
#                             else:
#                                 assert block_type == 1
#                                 block = block_builder.add_block(keys=block_idx, properties=np.asarray([[0]], dtype=np.int32),
#                                             components=[_components_idx(li), _components_idx(lj), np.array([+1,-1], np.int32).reshape(-1,1)] ) 
#------------------------------------------------
# ideally one would like to have blocks with different number of components - as in the COMMENTED out code above
# so the blocktype=1 will have a +-1 dimension on the components for permutation symmetry but not the other blockd
# Currently this raises an EquistoreError as the TensorMap requires all blocks to have the same components 

#Soln 1 - have the same number of components - so add a redundant dimension of perm_sym across ALL blocks -this code
#Soln 2 - append as component dimesnions for l1, l2 - the -1  
#--------------------------------------------------

                        else:
                            block = block_builder.blocks[block_idx]
                        
                        islice = slice(ki_offset,ki_offset+2*li+1)
                        jslice = slice(kj_offset,kj_offset+2*lj+1)
                        
                        if block_type == 1:
                            bdata = torch.stack([block_data_plus, block_data_minus])
#                             print(block_data.shape, block_data_plus[islice, jslice].shape, block_data[:2,islice, jslice].shape)
                            block.add_samples(labels=[(A,i,j)],data=bdata[:, islice, jslice].swapaxes(0,-2).reshape((1,2*li+1,2*lj+1,2,1)) )


                        else:
                            
                            bdata = torch.stack([block_data, torch.empty_like(block_data)])
                            print(bdata.shape, block_data.shape)
#                             print(block_data.shape)
                            block.add_samples(labels=[(A,i,j)], data=bdata[:,islice, jslice].swapaxes(0,-2).reshape((1,2*li+1,2*lj+1,2,1)) )
                           
                        
                        kj_offset += 2*lj+1
                    ki_offset += 2*li+1
                kj_base+=orbs_tot[aj]

            ki_base+=orbs_tot[ai]
    return block_builder.build()

## 

In [ ]:
def linear_init_weights(in_size, out_size):
    m=torch.nn.Linear(in_size, out_size)
    #m.weight.data.fill_(0)
    m.bias.data.fill_(0)
    return m 

class BlockModel(torch.nn.Module):
    def __init__(self, frames, block, feat, optim, layer_size=None):
        super().__init__()
        self.frames = frames
        self.nn = None
        self.layer_size = layer_size
        self.feature = feat
        self.values = block
        
        
    def initialize_model(self, seed):
        
        if seed is not None:
            torch.manual_seed(seed)
        inputSize = self.feature.values.shape[-1] #feature dimension 
        outputSize = self.block.values.shape[0] #block sample size 
        
        if self.layer_size is None: 
            self.nn = nn.Linear(inputSize, outputSize, bias = False)
            
        else:
#             self.layer1 = nn.Linear(10,10)
#             self.act1 = nn.TanH()
#             self.layer2 = nn.Linear(10,100)
#             self.act2 = nn.Sigmoid()
            self.nn = torch.nn.Sequential(
                linear_init_weights(inputSize, self.layer_size),
                torch.nn.Tanh(),
                linear_init_weights(self.layer_size, self.layer_size),
                torch.nn.Tanh(),
                linear_init_weights(self.layer_size, OutputSize),
            )
           
        
    def forward(self):
        if self.nn is None:
            raise Exception("call initialize_model first")
            
        pred = self.nn(self.feature.values)
#         pred = self.layer1(pred)
#         pred = self.act1(pred)
#         pred = self.layer2(pred)
#         pred = self.act2(pred)
#         pred = self.layer1(pred)
        return pred 

    def fit(loss_function, optimizer_type, lr, reg, nepochs = 5000):
        if optimizer_type == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = self.reg.item())
            threshold = 20000
            scheduler_threshold = 20000
            tol = 1e-3
            
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, scheduler_threshold, gamma = 0.5)
        best_state = copy.deepcopy(self.nn.state_dict())
        
        for epoch in tqdm(nepochs):
            optimizer.zero_grad()
            pred = self.nn(self.feature.values)
            loss = loss_function(self.values, pred, self.frames, )
            loss.backward()
            optimizer.step()
            scheduler.step()

In [ ]:
class LinearModel(torch.nn.Module):
    def __init__(self, coupled_blocks, features, weights=None, intercepts=None):
        super().__init__()
        self.coupled_blocks = coupled_blocks
        self.features = features
        self.weights = {}
        if weights==None:
            for idx_fock, block_fock in self.coupled_blocks:
                block_type, ai, ni, li, aj, nj, lj, L = idx_fock
                parity= (-1)**(li+lj+L)
                size = self.features.block(block_type=block_type, spherical_harmonics_l=L,inversion_sigma=parity, 
                                       species_center=ai, species_neighbor=aj).values.shape[2]
                #self.weights[idx_fock] = torch.nn.Parameter(torch.zeros(size, dtype=torch.float64))
                self.weights[idx_fock] = torch.nn.Parameter(torch.randn(size, dtype=torch.float64))
            
        else: 
            self.weights = weights
        
        self.intercepts = {}
        if intercepts is None:
            for idx_fock, block_fock in self.coupled_blocks:
                block_type, ai, ni, li, aj, nj, lj, L = idx_fock
                parity= (-1)**(li+lj+L)
                if L==0 and parity==1 and block_type==0:
                    self.intercepts[idx_fock] = torch.nn.Parameter(torch.randn(1, dtype=torch.float64))
                else:
                    self.intercepts[idx_fock] = 0
        else:
            self.intercepts = intercepts
         
    def forward(self, features):
        k = []
        pred_blocks = []
        for (idx, wts) in self.weights.items():
            #print(wts)
            block_type, ai, ni, li, aj, nj, lj, L = idx
            k.append(list(idx))
            parity= (-1)**(li+lj+L)
            X = features.block(block_type=block_type, spherical_harmonics_l=L,inversion_sigma=parity, 
                                   species_center=ai, species_neighbor=aj)
            X_new = torch.from_numpy(X.values.reshape(-1, X.values.shape[2]))
            #print(idx, wts.shape, X.values.shape, X_new.shape)
            Y = X_new @ wts + self.intercepts[idx]
            
            newblock = TensorBlock(
                        values=Y.reshape((-1, 2 * L + 1, 1)),
                        samples=X.samples,
                        components=[Labels(
                            ["mu"], np.asarray(range(-L, L + 1), dtype=np.int32).reshape(-1, 1)
                        )],
                        properties= Labels(["values"], np.asarray([[0]], dtype=np.int32))
                    )
            pred_blocks.append(newblock) 
        
        keys = Labels(('block_type', 'a_i', 'n_i', 'l_i', 'a_j', 'n_j', 'l_j', 'L'), np.asarray(k, dtype=np.int32))
        pred_fock = TensorMap(keys, pred_blocks)
        return(pred_fock)
        ### add direct eigenvalue prediction here as well
    
    def parameters(self):
        for idx, wts in self.weights.items():
            yield wts
        for idx, wts in self.intercepts.items():
            if type(wts) is not int:
                yield wts

In [403]:
#------------------------------------------------
#Soln 2 - append as sample dimension - the -1   
#--------------------------------------------------
from torch_hamiltonian_utils.torch_hamiltonians import _orbs_offsets, _components_idx, _components_idx_2d,_atom_blocks_idx

def blocks_to_dense(blocks, frames, orbs):
    """
    Converts a TensorMap containing matrix blocks in the uncoupled basis, `blocks` into dense matrices.
    Needs `frames` and `orbs` to reconstruct matrices in the correct order. See `dense_to_blocks` to understant
    the different types of blocks.
    """

    orbs_tot, orbs_offset =  _orbs_offsets(orbs)
    
    atom_blocks_idx = _atom_blocks_idx(frames, orbs_tot)
    
    # init storage for the dense hamiltonians
    dense = []        
    for f in frames:
        norbs = 0
        for ai in f.numbers:
            norbs += orbs_tot[ai]
        ham = torch.zeros(norbs, norbs)#, dtype=np.float64)
        dense.append(ham)

    # loops over block types
    for idx, block in blocks:
        cur_struct = 9999
        block_type, ai, ni, li, aj, nj, lj = tuple(idx)

        # offset of the orbital block within the pair block in the matrix
        ki_offset = orbs_offset[(ai,ni,li)]
        kj_offset = orbs_offset[(aj,nj,lj)]
        
        # loops over samples (structure, i, j)
        
        for (A,i,j), block_data in zip(block.samples, block.values): 
            if A<0:
                    continue 
            if A != cur_struct:
                cur_struct = A - 1
                ham = dense[cur_struct]
            # coordinates of the atom block in the matrix
            ki_base, kj_base = atom_blocks_idx[(cur_struct ,i,j)]
            islice = slice(ki_base+ki_offset, ki_base+ki_offset+2*li+1)
            jslice = slice(kj_base+kj_offset, kj_base+kj_offset+2*lj+1)

            # print(i, ni, li, ki_base, ki_offset)
            if block_type == 0:
                ham[islice, jslice] = block_data[:,:,0].reshape(2*li+1,2*lj+1)
                if ki_offset != kj_offset:
                    ham[jslice, islice] = block_data[:,:,0].reshape(2*li+1,2*lj+1).T
            elif block_type == 2:
                ham[islice, jslice] = block_data[:,:,0].reshape(2*li+1,2*lj+1)
                ham[jslice, islice] = block_data[:,:,0].reshape(2*li+1,2*lj+1).T   
                
            elif block_type == 1:

                block_idx_minus = np.where(np.logical_and(block.samples["structure"]==-A,  block.samples["center"]==i, block.samples["neighbor"]==j))
                print(idx, block_idx_minus, block.values[block_idx_minus].shape, li, lj)
#                 print(block.values[block_idx_minus][0,:,:,0].shape, block_data.shape), 
                block_data_minus = block.values[block_idx_minus][0,:,:,0].reshape(2*li+1,2*lj+1)  / np.sqrt(2)
                block_data_plus = block_data[:,:,0].reshape(2*li+1,2*lj+1)  / np.sqrt(2)
#                 print(cur_struct , block_data.shape)
                ham[islice, jslice] +=  block_data_plus
                ham[jslice, islice] +=  block_data_plus.T
                ham[islice, jslice] += block_data_minus
                ham[jslice, islice] += block_data_minus.T
                if ki_offset != kj_offset:
                    islice = slice(ki_base+kj_offset, ki_base+kj_offset+2*lj+1)
                    jslice = slice(kj_base+ki_offset, kj_base+ki_offset+2*li+1)
                    
                    ham[islice, jslice] += block_data_plus.T
                    ham[jslice, islice] += block_data_plus
                    
                    ham[islice, jslice] -= block_data_minus .T
                    ham[jslice, islice] -= block_data_minus
                
                    
    return dense

## 